In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [3]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# data generator for train and test data
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_dir = '/content/drive/MyDrive/10Classes/train'
test_dir = '/content/drive/MyDrive/10Classes/test'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Create the classifier model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Freeze the base model's layers
base_model.trainable = False

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model_vgg = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=5)

58889256/58889256 [==============================] - 0s 0us/step
Found 432 images belonging to 10 classes.
Found 111 images belonging to 10 classes.
Epoch 1/5
14/14 [==============================] - 248s 17s/step - loss: 29.0778 - accuracy: 0.3935
Epoch 2/5
14/14 [==============================] - 230s 16s/step - loss: 1.7586 - accuracy: 0.9028
Epoch 3/5
14/14 [==============================] - 230s 16s/step - loss: 0.3832 - accuracy: 0.9699
Epoch 4/5
14/14 [==============================] - 228s 16s/step - loss: 0.2287 - accuracy: 0.9815
Epoch 5/5
14/14 [==============================] - 230s 16s/step - loss: 0.0158 - accuracy: 0.9954


In [4]:

# Evaluate the model on the test set
y_pred = model.predict(test_generator)
y_pred = np.argmax(y_pred, axis=1)  # Get the index of the max value

# Get the true labels for the test set
y_true = test_generator.classes

# Calculate the confusion matrix and classification report
cm = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(cm)

class_names = list(train_generator.class_indices.keys())
report = classification_report(y_true, y_pred, target_names=class_names)
print('Classification Report:')
print(report)


4/4 [==============================] - 59s 14s/step
Confusion Matrix:
[[ 8  0  2  0  0  0  0  2  0  0]
 [ 3  9  0  0  0  0  0  0  0  0]
 [ 3  3  6  0  0  0  0  0  0  0]
 [ 0  0  0 11  0  0  0  0  0  1]
 [ 0  0  0  0  2  1  5  1  0  0]
 [ 0  0  1  0  0  8  0  0  0  0]
 [ 0  0  0  0  1  3  7  0  0  0]
 [ 1  0  0  0  1  0  0  8  0  0]
 [ 0  0  0  2  0  0  0  4  5  1]
 [ 0  0  0  0  0  0  0  0  0 12]]
Classification Report:
                            precision    recall  f1-score   support

001.Black_footed_Albatross       0.53      0.67      0.59        12
      002.Laysan_Albatross       0.75      0.75      0.75        12
       003.Sooty_Albatross       0.67      0.50      0.57        12
     004.Groove_billed_Ani       0.85      0.92      0.88        12
        005.Crested_Auklet       0.50      0.22      0.31         9
          006.Least_Auklet       0.67      0.89      0.76         9
       007.Parakeet_Auklet       0.58      0.64      0.61        11
     008.Rhinoceros_Auklet     

In [5]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import numpy as np
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [7]:

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_dir = '/content/drive/MyDrive/10Classes/train'
test_dir =  '/content/drive/MyDrive/10Classes/test'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

feature_extractor = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)
# feature_extractor = Model(inputs=model.input, outputs=model.get_layer('dense_1').output)

# Extract features from the train and test data
train_features = feature_extractor.predict(train_generator)
test_features = feature_extractor.predict(test_generator)


Found 432 images belonging to 10 classes.
Found 111 images belonging to 10 classes.
4/4 [==============================] - 59s 14s/step


In [8]:
train_features_flat = train_features.reshape(train_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)

In [9]:
train_features_flat.shape

(432, 25088)

In [10]:
test_features_flat.shape

(111, 25088)

In [11]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA
# from sklearn.manifold import TSNE

In [12]:
n_components = 32

# Apply PCA
pca = PCA(n_components=n_components)
train_features_reshaped = pca.fit_transform(train_features_flat)

# # Sparse PCA
# sparse_pca = SparsePCA(n_components=n_components, alpha=10)
# train_features_reshaped = sparse_pca.fit_transform(train_features_flat)

# tsne = TSNE(n_components = n_components)
# train_features_reshaped = tsne.fit_transform(train_features_flat)



In [13]:
n_components = 32

# Apply PCA
pca = PCA(n_components=n_components)
test_features_reshaped = pca.fit_transform(test_features_flat)

# sparse_pca = SparsePCA(n_components=n_components, alpha=10)
# test_features_reshaped = sparse_pca.fit_transform(test_features_flat)

# tsne = TSNE(n_components = n_components)
# test_features_reshaped = tsne.fit_transform(test_features_flat)

In [14]:
train_features_reshaped.shape

(432, 32)

In [15]:
test_features_reshaped.shape

(111, 32)

In [26]:
# ANN
from keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers

# model = Sequential()
# model.add(Flatten(input_shape=train_features.shape[1]))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(3, activation='sigmoid'))

model = keras.Sequential([
    layers.Input(shape=train_features_reshaped.shape[1]),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

sgd = tf.keras.optimizers.SGD(learning_rate=0.01)

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

train_labels = to_categorical(train_generator.labels, num_classes=10)
test_labels = to_categorical(test_generator.labels, num_classes=10)

# Train the model on the extracted features
model.fit(train_features_reshaped, train_labels, epochs=100, batch_size=32)

Epoch 1/100
14/14 [==============================] - 1s 4ms/step - loss: 96.0738 - accuracy: 0.1181
Epoch 2/100
14/14 [==============================] - 0s 4ms/step - loss: 46.6255 - accuracy: 0.1366
Epoch 3/100
14/14 [==============================] - 0s 5ms/step - loss: 29.5246 - accuracy: 0.1991
Epoch 4/100
14/14 [==============================] - 0s 3ms/step - loss: 19.8204 - accuracy: 0.2361
Epoch 5/100
14/14 [==============================] - 0s 2ms/step - loss: 13.6023 - accuracy: 0.3218
Epoch 6/100
14/14 [==============================] - 0s 2ms/step - loss: 9.8076 - accuracy: 0.4167
Epoch 7/100
14/14 [==============================] - 0s 2ms/step - loss: 7.0497 - accuracy: 0.4954
Epoch 8/100
14/14 [==============================] - 0s 2ms/step - loss: 5.3547 - accuracy: 0.5208
Epoch 9/100
14/14 [==============================] - 0s 2ms/step - loss: 3.6848 - accuracy: 0.6019
Epoch 10/100
14/14 [==============================] - 0s 3ms/step - loss: 2.5858 - accuracy: 0.6343
Epoc

In [27]:
y_pred = model.predict(test_features_reshaped)
y_pred = np.argmax(y_pred, axis=1)

y_true = test_generator.classes

cm = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(cm)

class_names = list(test_generator.class_indices.keys())
report = classification_report(y_true, y_pred, target_names=class_names)
print('Classification Report:')
print(report)

#model.save("/content/drive/My Drive/OCR/Stimulation/ANN.keras")

4/4 [==============================] - 0s 3ms/step
Confusion Matrix:
[[1 1 1 1 0 3 1 0 2 2]
 [1 1 2 2 0 0 1 0 2 3]
 [0 0 1 3 0 0 3 1 2 2]
 [1 1 0 1 0 4 1 1 1 2]
 [0 0 2 2 0 0 1 1 2 1]
 [2 3 2 1 1 0 0 0 0 0]
 [2 0 1 2 1 3 1 0 0 1]
 [0 0 2 2 1 2 0 1 0 2]
 [0 0 0 1 3 2 1 0 4 1]
 [0 1 2 0 0 1 1 5 1 1]]
Classification Report:
                            precision    recall  f1-score   support

001.Black_footed_Albatross       0.14      0.08      0.11        12
      002.Laysan_Albatross       0.14      0.08      0.11        12
       003.Sooty_Albatross       0.08      0.08      0.08        12
     004.Groove_billed_Ani       0.07      0.08      0.07        12
        005.Crested_Auklet       0.00      0.00      0.00         9
          006.Least_Auklet       0.00      0.00      0.00         9
       007.Parakeet_Auklet       0.10      0.09      0.10        11
     008.Rhinoceros_Auklet       0.11      0.10      0.11        10
      009.Brewer_Blackbird       0.29      0.33      0.31       

In [28]:
# Random Forest
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_labels_rf = train_generator.labels
test_labels_rf = test_generator.labels

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(train_features_reshaped, train_labels_rf)

y_pred = rf_classifier.predict(test_features_reshaped)

In [29]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels_rf, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Generate and print a classification report
report = classification_report(test_labels_rf, y_pred)
print("Classification Report:\n", report)

# Generate and print a confusion matrix
conf_matrix = confusion_matrix(test_labels_rf, y_pred)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.10
Classification Report:
               precision    recall  f1-score   support

           0       0.09      0.17      0.12        12
           1       0.12      0.17      0.14        12
           2       0.29      0.42      0.34        12
           3       0.05      0.08      0.06        12
           4       0.00      0.00      0.00         9
           5       0.00      0.00      0.00         9
           6       0.00      0.00      0.00        11
           7       0.00      0.00      0.00        10
           8       0.11      0.08      0.10        12
           9       0.00      0.00      0.00        12

    accuracy                           0.10       111
   macro avg       0.07      0.09      0.08       111
weighted avg       0.07      0.10      0.08       111

Confusion Matrix:
 [[2 2 1 1 0 0 2 1 1 2]
 [1 2 2 4 0 0 1 1 1 0]
 [2 0 5 1 0 0 0 1 2 1]
 [5 2 0 1 0 0 1 0 1 2]
 [1 1 3 3 0 0 0 0 1 0]
 [2 3 2 2 0 0 0 0 0 0]
 [1 4 0 1 1 0 0 0 1 3]
 [2 0 0 3 1 0 0 0 0 4]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
